# Manual Tool Testing for LangGraph Agent

This notebook allows manual testing of all tools to verify they work correctly before using them in the agent.

## Setup: Load Environment and Import Tools

In [1]:
# Add project root to path to ensure imports work
import sys
from pathlib import Path

# Get project root (parent of agent folder)
project_root = Path.cwd().parent
print(f"Project root: {project_root}")

# Add to path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print("Added project root to sys.path")

Project root: /Users/ayesha/Projects/apertureDB Demo
Added project root to sys.path


In [2]:
# Load environment variables
from dotenv import load_dotenv
import os

# Load .env from project root
env_path = project_root / ".env"
print(f"Loading .env from: {env_path}")
print(f".env exists: {env_path.exists()}")

load_dotenv(env_path)

# Verify keys are loaded
print(f"\nEnvironment variables loaded:")
print(f"  APERTUREDB_KEY: {'✅ Set' if os.getenv('APERTUREDB_KEY') else '❌ Not set'}")
print(f"  GOOGLE_API_KEY: {'✅ Set' if os.getenv('GOOGLE_API_KEY') else '❌ Not set'}")
print(f"  APERTUREDB_KEY length: {len(os.getenv('APERTUREDB_KEY', ''))} chars")

Loading .env from: /Users/ayesha/Projects/apertureDB Demo/.env
.env exists: True

Environment variables loaded:
  APERTUREDB_KEY: ✅ Set
  GOOGLE_API_KEY: ✅ Set
  APERTUREDB_KEY length: 128 chars


In [3]:
# Import all tools
from tools.tools import (
    search_talks_by_filters,
    search_talks_semantically,
    analyze_speaker_activity,
    get_talk_details,
    find_similar_content,
    analyze_topics_and_trends
)

print("✅ All tools imported successfully!")

✅ All tools imported successfully!


## Test 1: Search Talks by Filters (Top 5 Most Viewed)

In [4]:
# Test the exact query the agent tried
print("Testing: search_talks_by_filters")
print("Query: Top 5 most viewed talks\n")

result = search_talks_by_filters(
    limit=5,
    sort_by="views",
    sort_order="desc"
)

print("Result:")
print(result)

Testing: search_talks_by_filters
Query: Top 5 most viewed talks



/var/folders/p7/rnmpzv5s403_kckd36fzc8040000gn/T/ipykernel_62544/2830651348.py:5: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = search_talks_by_filters(


TypeError: BaseTool.__call__() got an unexpected keyword argument 'limit'

In [6]:
# Test the exact query the agent tried
print("Testing: search_talks_by_filters")
print("Query: Top 5 most viewed talks\n")

result3 = search_talks_by_filters.invoke({
    "company_name": "Google",
    "sort_by": "views",
    "sort_order": "desc",
    "limit": 8
})

print("Result:")
print(result3)

Testing: search_talks_by_filters
Query: Top 5 most viewed talks

Result:
{'results': [{'talk_id': 'f59b6283-69e7-5630-8789-4fe2bf0fca8c', 'title': 'Agentic AI: Unlocking Emergent Behavior in LLMs for Adaptive Workflow Automation', 'speaker': 'Patrick Marlow', 'company': 'Google', 'views': 2372, 'published_date': '2024-10-31', 'youtube_url': 'https://www.youtube.com/watch?v=GwQi33fmexU', 'event': 'TMLS 2024', 'category': 'Introduction to MLOps and GenAI', 'tech_level': 4, 'track': 'Applied Case Studies', 'abstract': 'Explore the emergent capabilities of "agentic" AI, where agents combine LLMs, reasoning loops, and tools to tackle complex workflows beyond the capabilities of LLMs alone. This session examines techni...'}, {'talk_id': '3bec210f-b106-5ff7-ab2c-3b467ff012d9', 'title': 'Optimizing AI/ML Workflows on Kubernetes: Advanced Techniques and Integration', 'speaker': 'Anu Reddy', 'company': 'Google', 'views': 26, 'published_date': '2024-12-11', 'youtube_url': 'https://www.youtube.com

## Test 2: Direct ApertureDB Connection Test

In [ ]:
# Test ApertureDB connection directly
from tools.utils import get_db_connector

print("Testing direct ApertureDB connection...\n")

try:
    con = get_db_connector()
    print("✅ Connection successful!\n")
    
    # Try a simple query
    print("Testing simple query: Count all Talk entities\n")
    query = [{
        "FindEntity": {
            "with_class": "Talk",
            "results": {
                "count": True
            }
        }
    }]
    
    response = con.query(query)
    print("Response:")
    print(response)
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 3: Query Top 5 Talks Directly with ApertureDB

In [ ]:
# Test the exact query that search_talks_by_filters would generate
from tools.utils import get_db_connector

print("Testing: Direct ApertureDB query for top 5 most viewed talks\n")

try:
    con = get_db_connector()
    
    query = [{
        "FindEntity": {
            "with_class": "Talk",
            "constraints": {
                "yt_views": [">=", 0]  # Ensure yt_views exists
            },
            "results": {
                "all_properties": True,
                "limit": 5
            },
            "sort": {
                "key": "yt_views",
                "order": "descending"
            }
        }
    }]
    
    print("Query:")
    import json
    print(json.dumps(query, indent=2))
    print("\nExecuting query...\n")
    
    response, _ = con.query(query)
    
    print("Response:")
    print(json.dumps(response, indent=2))
    
    # Extract and display talks
    if response and len(response) > 0:
        find_result = response[0].get('FindEntity', {})
        entities = find_result.get('entities', [])
        
        if entities:
            print(f"\n✅ Found {len(entities)} talks:\n")
            for i, talk in enumerate(entities, 1):
                title = talk.get('talk_title', 'N/A')
                views = talk.get('yt_views', 'N/A')
                speaker = talk.get('speaker_name', 'N/A')
                print(f"{i}. {title}")
                print(f"   Speaker: {speaker}")
                print(f"   Views: {views:,}" if isinstance(views, int) else f"   Views: {views}")
                print()
        else:
            print("❌ No entities found in response")
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 4: Check Database Schema

In [ ]:
# Check what properties are available on Talk entities
from tools.utils import get_db_connector

print("Checking database schema...\n")

try:
    con = get_db_connector()
    
    query = [{"GetSchema": {}}]
    response, _ = con.query(query)
    
    import json
    print("Schema Response:")
    print(json.dumps(response, indent=2))
    
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

## Test 5: Search Talks Semantically

In [ ]:
# Test semantic search
print("Testing: search_talks_semantically")
print("Query: AI agents with memory\n")

result = search_talks_semantically(
    query="AI agents with memory",
    search_type="transcripts",
    limit=3
)

print("Result:")
print(result)

## Test 6: Get Talk Details

In [ ]:
# First get a talk title from the top viewed talks
# Then test get_talk_details

print("Testing: get_talk_details\n")

# Use a known talk title (replace with actual title from Test 3 results)
result = get_talk_details(
    talk_title="Multimodal Agents You Can Deploy Anywhere",  # Example
    include_transcript=False
)

print("Result:")
print(result)

## Summary

Run all cells above to test each tool individually and identify any issues.